In [1]:
# First, as always, we install all the packages we will use
import pandas as pd
import numpy as numpy
import json
!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Collecdone
done

# All requested packages already installed.

done

# All requested packages already installed.



In [2]:
#import requests to get data from wikipedia
import requests
toronto_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print('Data downloaded!')

Data downloaded!


In [6]:
# I will be using the BeautifulSoup package to scrap the table from Wikipedia
from bs4 import BeautifulSoup
soup = BeautifulSoup(toronto_data.content, "html.parser")

In [8]:
# Lets take an initial llok at the data in the table
table = soup.find_all('table')[0] 
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East Y

Notice how all the relevant data is in the th,tr,td labelled keys, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [15]:
post_code = []
boroughs = []
neighborhoods = []

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        post_code.append(cells[0].find(text=True))
        boroughs.append(cells[1].find(text=True))
        neighborhoods.append(cells[2].find(text=True))

The next task is essentially transforming this data of nested Python list into a pandas dataframe. So let's start by creating an empty dataframe and then seeing what we have thus far

In [120]:
# define the dataframe columns
column_names = ['Postal Code','Borough', 'Neighborhood'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [121]:
df

,Postal Code,Borough,Neighborhood


In [122]:
# Now we assign the corresponding lists to the appropriate column.
df['Postal Code'] = post_code
df['Borough'] = boroughs
df['Neighborhood'] = neighborhoods
df

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
175,M5Z\n,Not assigned\n,\n
176,M6Z\n,Not assigned\n,\n
177,M7Z\n,Not assigned\n,\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


In [123]:
#Lets remove all the '\n' new line characters.
df = df.replace('\n','', regex=True)
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [124]:
# Drop all rows that do not have a borough assigned.
df = df[df.Borough != 'Not assigned' ]

In [125]:
#reset the index 
df.reset_index(drop=True, inplace=True)

In [126]:
# 
df.groupby(['Postal Code'])

In [130]:
#We can test for a unique Postal Code but printing all lines of a randomly chosen Postal Code
df[df.isin(["M4B"]).any(axis=1)]

,Postal Code,Borough,Neighborhood
8,M4B,East York,"Parkview Hill, Woodbine Gardens"


In [133]:
#lets print the shape of the dtaframe to identify the number of rowas as requested.
df.shape

(103, 3)